In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
avocado = pd.read_csv("dataset/avocado_2015_to_2020.csv")

In [7]:
avocado.head()

,AveragePrice,Total Volume,Small/Medium Hass,Large Hass,Extra Large Hass,Total Bags,Small Bags,Large Bags,XLarge Bags,Type,Year,Region,Month
0,1.75,27365.89,9307.34,3844.81,615.28,13598.46,13061.10,537.36,0.0,Organic,2015,Southeast,Jan
1,1.49,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.0,Organic,2015,Chicago,Jan
2,1.68,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.0,Organic,2015,HarrisburgScranton,Jan
3,1.52,54956.80,3013.04,35456.88,1561.70,14925.18,11264.80,3660.38,0.0,Conventional,2015,Pittsburgh,Jan
4,1.64,1505.12,1.27,1129.50,0.00,374.35,186.67,187.68,0.0,Organic,2015,Boise,Jan


In [5]:
avocado = avocado.rename(columns={"4046": "Small/Medium Hass", "4225": "Large Hass", "4770": "Extra Large Hass"})

In [31]:
avocado.head()

,AveragePrice,Total Volume,Small/Medium Hass,Large Hass,Extra Large Hass,Total Bags,Small Bags,Large Bags,XLarge Bags,Type,Year,Region,Month
0,1.75,27365.89,9307.34,3844.81,615.28,13598.46,13061.10,537.36,0.0,Organic,2015,Southeast,Jan
1,1.49,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.0,Organic,2015,Chicago,Jan
2,1.68,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.0,Organic,2015,HarrisburgScranton,Jan
3,1.52,54956.80,3013.04,35456.88,1561.70,14925.18,11264.80,3660.38,0.0,Conventional,2015,Pittsburgh,Jan
4,1.64,1505.12,1.27,1129.50,0.00,374.35,186.67,187.68,0.0,Organic,2015,Boise,Jan


In [39]:
month_order = avocado["Month"].unique()
year_order = avocado["Year"].unique()

In [52]:
average_total_per_year_month_region = avocado.groupby(["Year", "Month", "Region"])\
                        .agg({"AveragePrice": "mean", "Total Volume": "sum"})\
                        .reindex(month_order, level=1)

In [53]:
average_total_per_year_month_region

AveragePrice  Total Volume
Year Month Region                                         
2015 Jan   Albany                   1.508750  1.765180e+05
           Atlanta                  1.456250  1.728320e+06
           BaltimoreWashington      1.251250  2.883524e+06
           Boise                    1.330000  2.743618e+05
           Boston                   1.537500  1.748752e+06
...                                      ...           ...
2020 Apr   Syracuse                 1.430000  3.321004e+05
           Tampa                    1.188333  1.993183e+06
           TotalUS                  1.380000  1.549141e+08
           West                     1.426667  2.589030e+07
           WestTexNewMexico         1.291667  3.545293e+06

[3456 rows x 2 columns]